<a href="https://colab.research.google.com/github/theCodingEvolution/retinography-using-VGG16/blob/master/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input,Lambda,Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16

from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
#resize all the image to this (For VGG16)
IMAGE_SIZE= [224,224]

In [3]:
#resize all the image to this (For VGG16)
IMAGE_SIZE= [224,224]

In [4]:
# ADDING preprocessing layer to thr frony of VGG_16
vgg=VGG16(input_shape=IMAGE_SIZE + [3],weights="imagenet",include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [5]:
# DONT train existing weights
for layer in vgg.layers:
    layer.trainable= False


In [6]:
# usefull for getting number if classes
folders=glob('/content/drive/MyDrive/Datasets(Retinography)/Train/*') 
print(folders)
test_folder=glob('/content/drive/MyDrive/Datasets(Retinography)/Test/*')
print(test_folder)

['/content/drive/MyDrive/Datasets(Retinography)/Train/_NoDR', '/content/drive/MyDrive/Datasets(Retinography)/Train/DR']
['/content/drive/MyDrive/Datasets(Retinography)/Test/_NoDR', '/content/drive/MyDrive/Datasets(Retinography)/Test/DR']


In [7]:
x=Flatten()(vgg.output)
#x=Dense(1000,activstion='relu'),(x)
prediction=Dense(len(folders), activation='softmax')(x)

In [8]:
# create a model object
model=Model(inputs=vgg.input, outputs=prediction)

# model summery
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
#cost and optimization functions for the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
              )


In [10]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [11]:
training_set=train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets(Retinography)/Train',
                                               target_size= (224,224),
                                               batch_size= 64,
                                               class_mode='categorical')
val_set=test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets(Retinography)/Val',
                                           target_size=(224,224),
                                           batch_size=64,
                                           class_mode= 'categorical')

Found 3424 images belonging to 2 classes.
Found 1426 images belonging to 2 classes.


# FIT THE  MODEL

In [ ]:
r= model.fit(
    training_set,validation_data=val_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(val_set)
)

Epoch 1/5
54/54 [==============================] - ETA: 0s - loss: 0.4303 - accuracy: 0.8586 

In [ ]:
#loss
plt.plot(r.history['loss'], label='train_loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.savefig('AccVal_loss')

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Datasets(Retinography)/Test',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

In [ ]:
model.evaluate(test_set)